<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/FinRL_PortfolioAllocation_NeurIPS_2020.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Portfolio Allocation

Tutorials to use OpenAI DRL to perform portfolio allocation in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog is based on our paper: FinRL: A Deep Reinforcement Learning Library for Automated Stock Trading in Quantitative Finance, presented at NeurIPS 2020: Deep RL Workshop.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-portfolio-allocation-9b417660c7cd
* Please report any issues to our Github: https://github.com/AI4Finance-Foundation/FinRL/issues
* **Pytorch Version**



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for portfolio alloacation. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A represents the weight of a stock in the porfolio: a ∈ (-1,1). Assume our stock pool includes N stocks, we can use a list [a<sub>1</sub>, a<sub>2</sub>, ... , a<sub>N</sub>] to determine the weight for each stock in the porfotlio, where a<sub>i</sub> ∈ (-1,1), a<sub>1</sub>+ a<sub>2</sub>+...+a<sub>N</sub>=1. For example, "The weight of AAPL in the portfolio is 10%." is [0.1 , ...].

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [111]:
## install finrl library
!pip install wrds
!pip install swig
!pip install -q condacolab

!pip install 'shimmy>=2.0'
!pip install pandas_market_calendars
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

zsh:1: command not found: apt-get
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-req-build-01fjwjnm
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-req-build-01fjwjnm
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit d25d902a6de54931a329adc38a2663e8f576adc4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-install-j23zyn2u/elegantrl_6194f6e594394cb2a633ef2edb0e78f4
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-install-j23zyn2u/elegantrl_6


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them.
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [205]:
# ===========================
# Suppress Warnings & Backend Setup
# ===========================
import warnings
warnings.filterwarnings("ignore")

import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for environments without GUI

# ===========================
# Standard Libraries
# ===========================
import os
import sys
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For Jupyter Notebooks (optional, safe to keep)
%matplotlib inline

# ===========================
# FinRL Imports
# ===========================
from finrl import config, config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import (
    backtest_stats, 
    backtest_plot, 
    get_daily_return, 
    get_baseline,
    convert_daily_return_to_pyfolio_ts
)
from finrl.meta.data_processor import DataProcessor
from finrl.meta.data_processors.processor_yahoofinance import YahooFinanceProcessor
from finrl.main import check_and_make_directories
from finrl.config import (
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
)

# ===========================
# Create Necessary Directories
# ===========================
check_and_make_directories([
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR
])

# ===========================
# Custom Library Path
# ===========================
sys.path.append("../FinRL-Library")

<a id='1.4'></a>
## 2.4. Create Folders

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).


In [114]:
TRAIN_START_DATE = '2016-02-02'
TRAIN_END_DATE = '2023-04-04'
TRADE_START_DATE = '2023-04-05'
TRADE_END_DATE = '2025-04-10'

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [206]:
# import pandas as pd
# import numpy as np
# from finrl.meta.preprocessor.preprocessors import FeatureEngineer

def process_csv_to_features(csv_path, lookback=252):
    # Step 1: Load data
    df = pd.read_csv(csv_path)

    # Step 2: Identify 5-day and 7-day tickers
    day_values_per_tic = df.groupby('tic')['day'].apply(lambda x: sorted(x.unique())).reset_index()
    day_values_per_tic.columns = ['tic', 'unique_days']
    tics_5day = day_values_per_tic[day_values_per_tic['unique_days'].apply(lambda x: x == list(range(5)))]['tic']
    tics_7day = day_values_per_tic[day_values_per_tic['unique_days'].apply(lambda x: x == list(range(7)))]['tic']

    # Step 3: Filter tickers
    df_5day_full = df[df['tic'].isin(tics_5day)]
    df_7day_full = df[df['tic'].isin(tics_7day)]

    # Step 4: Apply technical indicators
    fe = FeatureEngineer(use_technical_indicator=True, use_turbulence=False, user_defined_feature=False)
    df_5day_full = fe.preprocess_data(df_5day_full)
    df_7day_full = fe.preprocess_data(df_7day_full)

    # Step 5: Combine and clean
    df = pd.concat([df_5day_full, df_7day_full], ignore_index=False)
    df.index = range(len(df))
    df['date'] = pd.to_datetime(df['date'])
    df = df[df.groupby('date')['date'].transform('count') > 1]
    df = df.sort_values('date').reset_index(drop=True)

    # Step 6: Prepare for covariance matrix computation
    df = df.sort_values(['date', 'tic'], ignore_index=True)
    df.index = df.date.factorize()[0]  # Re-index based on unique date

    cov_list = []
    return_list = []
    unique_indices = df.index.unique()

    for i in range(lookback, len(unique_indices)):
        data_lookback = df.loc[i - lookback:i, :]
        price_lookback = data_lookback.pivot_table(index='date', columns='tic', values='close')
        return_lookback = price_lookback.pct_change().dropna()
        return_list.append(return_lookback)
        cov_list.append(return_lookback.cov().values)

    # Step 7: Merge covariance matrix and return series back
    df_cov = pd.DataFrame({
        'date': df.date.unique()[lookback:], 
        'cov_list': cov_list, 
        'return_list': return_list
    })
    df = df.merge(df_cov, on='date')
    df = df.sort_values(['date', 'tic']).reset_index(drop=True)

    return df


In [207]:
processed = process_csv_to_features('data.csv')

Successfully added technical indicators
Successfully added technical indicators


In [208]:
processed.head()

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2016-02-02,109.32,109.38,109.1301,109.23,4835115.0,agg,1,0.207871,109.414436,108.155564,56.401620,128.048919,18.311124,108.540000,108.509083,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
1,2016-02-02,91.34,91.36,91.3400,91.34,1627284.0,bil,1,-0.003626,91.386806,91.327194,46.986934,-124.094203,12.399460,91.360333,91.364333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
2,2016-02-02,372.93,374.41,371.1700,371.33,6817.0,btcusd,1,-12.397434,422.125588,352.168412,45.804685,-84.272928,40.704445,405.695000,419.803333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
3,2016-02-02,108.09,108.18,107.3500,107.92,6656018.0,gld,1,1.135824,108.620100,102.477820,56.790972,143.671556,35.269843,104.479307,103.650320,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
4,2016-02-02,190.16,191.97,189.5400,191.96,182564890.0,spy,1,-2.744645,199.106331,183.296129,43.397377,-51.889822,20.952982,195.478153,201.393583,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.


## Training data split: 2009-01-01 to 2020-07-01

In [188]:
train = data_split(processed, TRAIN_START_DATE, TRAIN_END_DATE)
trade = data_split(processed, TRADE_START_DATE, TRADE_END_DATE)

In [189]:
train.head()

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2016-02-02,109.32,109.38,109.1301,109.23,4835115.0,agg,1,0.207871,109.414436,108.155564,56.401620,128.048919,18.311124,108.540000,108.509083,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
0,2016-02-02,91.34,91.36,91.3400,91.34,1627284.0,bil,1,-0.003626,91.386806,91.327194,46.986934,-124.094203,12.399460,91.360333,91.364333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
0,2016-02-02,372.93,374.41,371.1700,371.33,6817.0,btcusd,1,-12.397434,422.125588,352.168412,45.804685,-84.272928,40.704445,405.695000,419.803333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
0,2016-02-02,108.09,108.18,107.3500,107.92,6656018.0,gld,1,1.135824,108.620100,102.477820,56.790972,143.671556,35.269843,104.479307,103.650320,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
0,2016-02-02,190.16,191.97,189.5400,191.96,182564890.0,spy,1,-2.744645,199.106331,183.296129,43.397377,-51.889822,20.952982,195.478153,201.393583,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...


In [136]:
train.head()

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2016-02-02,109.32,109.38,109.1301,109.23,4835115.0,agg,1,0.207871,109.414436,108.155564,56.401620,128.048919,18.311124,108.540000,108.509083,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
0,2016-02-02,91.34,91.36,91.3400,91.34,1627284.0,bil,1,-0.003626,91.386806,91.327194,46.986934,-124.094203,12.399460,91.360333,91.364333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
0,2016-02-02,372.93,374.41,371.1700,371.33,6817.0,btcusd,1,-12.397434,422.125588,352.168412,45.804685,-84.272928,40.704445,405.695000,419.803333,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
0,2016-02-02,108.09,108.18,107.3500,107.92,6656018.0,gld,1,1.135824,108.620100,102.477820,56.790972,143.671556,35.269843,104.479307,103.650320,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...
0,2016-02-02,190.16,191.97,189.5400,191.96,182564890.0,spy,1,-2.744645,199.106331,183.296129,43.397377,-51.889822,20.952982,195.478153,201.393583,"[[5.736252007431309e-06, 6.710303377777988e-09...",tic agg bil btcusd ...


## Environment for Portfolio Allocation


In [137]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step


    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]


    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions)
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state

    def render(self, mode='human'):
        return self.state

    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output


    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [192]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 9, State Space: 9


In [201]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4

}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']


In [194]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [195]:
# initialize
agent = DRLAgent(env = env_train)

### Model 1: **A2C**


In [196]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device


In [197]:
trained_a2c = agent.train_model(model=model_a2c,
                                tb_log_name='a2c',
                                total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 1083      |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -12.8     |
|    explained_variance | 1.79e-07  |
|    learning_rate      | 0.0002    |
|    n_updates          | 99        |
|    policy_loss        | 7.61e+07  |
|    reward             | 2050692.1 |
|    std                | 0.999     |
|    value_loss         | 4.34e+13  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 969       |
|    iterations         | 200       |
|    time_elapsed       | 1         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -12.8     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0002    |
|    n_updat

In [198]:
trained_a2c.save('trained_models/a2c.zip')

### Model 2: **PPO**


In [145]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cpu device


In [146]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=80000)

begin_total_asset:1000000
end_total_asset:3094027.6597703025
Sharpe:  1.0800524793526054
----------------------------------
| time/              |           |
|    fps             | 2274      |
|    iterations      | 1         |
|    time_elapsed    | 0         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 1272788.6 |
----------------------------------
begin_total_asset:1000000
end_total_asset:2823765.8780237655
Sharpe:  1.0190661908886416
---------------------------------------
| time/                   |           |
|    fps                  | 2066      |
|    iterations           | 2         |
|    time_elapsed         | 1         |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -12.8     |
|    explained_variance   | -2.38e-07 |
|    learning_rate        |

In [147]:
trained_ppo.save('trained_models/ppo.zip')

### Model 3: **DDPG**


In [148]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [173]:
trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=50000)

KeyboardInterrupt: 

In [ ]:
trained_ddpg.save('trained_models/ddpg.zip')

### Model 4: **SAC**


In [151]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0003, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [152]:
trained_sac = agent.train_model(model=model_sac,
                             tb_log_name='sac',
                             total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:3654744.626536127
Sharpe:  1.146103019070211
begin_total_asset:1000000
end_total_asset:3719214.3125598216
Sharpe:  1.1565868781979913
begin_total_asset:1000000
end_total_asset:3719214.3125598216
Sharpe:  1.1565868781979913
begin_total_asset:1000000
end_total_asset:3719214.3125598216
Sharpe:  1.1565868781979913
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 199       |
|    time_elapsed    | 36        |
|    total_timesteps | 7220      |
| train/             |           |
|    actor_loss      | -7.42e+07 |
|    critic_loss     | 2.09e+12  |
|    ent_coef        | 0.921     |
|    ent_coef_loss   | 10.6      |
|    learning_rate   | 0.0003    |
|    n_updates       | 7119      |
|    reward          | 3719214.2 |
----------------------------------
begin_total_asset:1000000
end_total_asset:3719214.3125598216
Sharpe:  1.1565868781979913
begin_total_asset:1000000
end_to

In [153]:
trained_sac.save('trained_models/sac.zip')

### Model 5: **TD3**


In [154]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100,
              "buffer_size": 1000000,
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [174]:
trained_td3 = agent.train_model(model=model_td3,
                             tb_log_name='td3',
                             total_timesteps=30000)

KeyboardInterrupt: 

In [156]:
trained_td3.save('trained_models/td3.zip')

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the A2C model to trade Dow jones 30 stocks.

In [157]:
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

trained_a2c = A2C.load("trained_models/a2c")
trained_ddpg = DDPG.load("trained_models/ddpg")
trained_ppo = PPO.load("trained_models/ppo")
trained_td3 = TD3.load("trained_models/td3")
trained_sac = SAC.load("trained_models/sac")

In [210]:
def train_and_predict_drl(df, 
                        train_start_date, train_end_date, 
                        trade_start_date, trade_end_date, 
                        model_name,
                        output_return_csv,
                        output_action_csv,
                        total_timesteps=None):
    """
    Train, save, reload, and predict using DRL models (A2C, PPO, DDPG, SAC, TD3).
    """

    import os
    from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
    from finrl.agents.stablebaselines3.models import DRLAgent
    from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

    # === Fixed Model Hyperparameters ===
    model_configs = {
        "a2c": {
            "params": {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002},
            "timesteps": 50_000
        },
        "ppo": {
            "params": {"n_steps": 2048, "ent_coef": 0.005, "learning_rate": 0.0001, "batch_size": 128},
            "timesteps": 80_000
        },
        "ddpg": {
            "params": {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001},
            "timesteps": 50_000
        },
        "sac": {
            "params": {"batch_size": 128, "buffer_size": 100000, "learning_rate": 0.0003, "learning_starts": 100, "ent_coef": "auto_0.1"},
            "timesteps": 50_000
        },
        "td3": {
            "params": {"batch_size": 100, "buffer_size": 1_000_000, "learning_rate": 0.001},
            "timesteps": 30_000
        }
    }

    # === Mapping model_name to class ===
    model_class_map = {
        "a2c": A2C,
        "ppo": PPO,
        "ddpg": DDPG,
        "sac": SAC,
        "td3": TD3
    }

    if model_name not in model_configs:
        raise ValueError(f"Model '{model_name}' is not supported. Choose from {list(model_configs.keys())}")

    config = model_configs[model_name]
    model_params = config["params"]
    timesteps = total_timesteps if total_timesteps else config["timesteps"]

    # === Data Split ===
    train_data = data_split(df, train_start_date, train_end_date)
    trade_data = data_split(df, trade_start_date, trade_end_date)

    stock_dimension = len(train_data.tic.unique())
    state_space = stock_dimension
    print(f"[INFO] Training {model_name.upper()} | Timesteps: {timesteps}")

    # === Environment Setup ===
    env_kwargs = {
        "hmax": 100,
        "initial_amount": 1_000_000,
        "transaction_cost_pct": 0.001,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": INDICATORS,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4
    }

    # === Train Model ===
    e_train_gym = StockPortfolioEnv(df=train_data, **env_kwargs)
    env_train, _ = e_train_gym.get_sb_env()

    agent = DRLAgent(env=env_train)
    model = agent.get_model(model_name=model_name, model_kwargs=model_params)
    trained_model = agent.train_model(model=model, tb_log_name=model_name, total_timesteps=timesteps)

    # === Save Model ===
    os.makedirs("trained_models", exist_ok=True)
    model_save_path = f"trained_models/{model_name}.zip"
    trained_model.save(model_save_path)
    print(f"[INFO] Model saved at {model_save_path}")

    # === Reload Model Automatically ===
    ReloadedModelClass = model_class_map[model_name]
    reloaded_model = ReloadedModelClass.load(model_save_path)
    print(f"[INFO] Model '{model_name.upper()}' reloaded from {model_save_path}")

    # === Predict ===
    e_trade_gym = StockPortfolioEnv(df=trade_data, **env_kwargs)
    df_daily_return, df_actions = DRLAgent.DRL_prediction(model=reloaded_model, environment=e_trade_gym)

    # === Save Outputs ===
    df_daily_return.to_csv(output_return_csv, index=False)
    df_actions.to_csv(output_action_csv, index=False)

    print(f"[INFO] Outputs saved:")
    print(f" - Daily Return: {output_return_csv}")
    print(f" - Actions     : {output_action_csv}")

    return df_daily_return, df_actions


In [214]:
df_daily_return_1, df_actions_1 = train_and_predict_drl(
    df = processed,
    train_start_date = TRAIN_START_DATE,
    train_end_date = TRAIN_END_DATE,
    trade_start_date = TRADE_START_DATE,
    trade_end_date = TRADE_END_DATE,
    model_name = 'a2c',
    output_return_csv = 'df_daily_return_1.csv',
    output_action_csv = 'df_actions_1.csv'
)


[INFO] Training A2C | Timesteps: 50000
{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device
-------------------------------------
| time/                 |           |
|    fps                | 1320      |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -12.7     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0002    |
|    n_updates          | 99        |
|    policy_loss        | 6.96e+07  |
|    reward             | 1956946.4 |
|    std                | 0.996     |
|    value_loss         | 4.03e+13  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 1235      |
|    iterations         | 200       |
|    time_elapsed       | 0         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entrop

In [216]:
df_daily_return_1.head()
df_actions_1.head()

,agg,bil,btcusd,gld,spy,vb,vnq,vo,vwo
date,,,,,,,,,
2023-04-05,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
2023-04-06,0.173956,0.068817,0.063995,0.130088,0.074334,0.173956,0.173956,0.076903,0.063995
2023-04-10,0.173956,0.068817,0.063995,0.130088,0.074334,0.173956,0.173956,0.076903,0.063995
2023-04-11,0.173956,0.068817,0.063995,0.130088,0.074334,0.173956,0.173956,0.076903,0.063995
2023-04-12,0.173322,0.069989,0.063761,0.132204,0.073280,0.173322,0.173322,0.077039,0.063761


In [199]:
# trade = data_split(df, TRADE_START_DATE, TRADE_END_DATE)
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [200]:
df_daily_return_1, df_actions_1 = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

df_daily_return_1.to_csv('df_daily_return_1_ori.csv')

begin_total_asset:1000000
end_total_asset:1194539.9683249586
Sharpe:  0.9781944121670434
hit end!


In [159]:
trade.shape

(4545, 18)

In [160]:
df_daily_return_1, df_actions_1 = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)
df_daily_return_2, df_actions_2 = DRLAgent.DRL_prediction(model=trained_ddpg,
                        environment = e_trade_gym)
df_daily_return_3, df_actions_3 = DRLAgent.DRL_prediction(model=trained_ppo,
                        environment = e_trade_gym)
df_daily_return_4, df_actions_4 = DRLAgent.DRL_prediction(model=trained_sac,
                        environment = e_trade_gym)
df_daily_return_5, df_actions_5 = DRLAgent.DRL_prediction(model=trained_td3,
                        environment = e_trade_gym)

/opt/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1371133.7426636913
Sharpe:  1.211784917270324
hit end!


/opt/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1389576.5969557953
Sharpe:  1.3415812321615228
hit end!


/opt/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1313117.5508216247
Sharpe:  1.1890414136527794
hit end!


/opt/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1409861.4051000425
Sharpe:  1.38241719833914
hit end!


/opt/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1401228.280985471
Sharpe:  1.4658790003944726
hit end!


In [161]:
df_daily_return_1.head()

,date,daily_return
0,2023-04-05,0.000000
1,2023-04-06,0.000290
2,2023-04-10,0.010468
3,2023-04-11,0.006661
4,2023-04-12,-0.003919


In [162]:
df_daily_return_1.to_csv('df_daily_return_1.csv')
df_daily_return_2.to_csv('df_daily_return_2.csv')
df_daily_return_3.to_csv('df_daily_return_3.csv')
df_daily_return_4.to_csv('df_daily_return_4.csv')
df_daily_return_5.to_csv('df_daily_return_5.csv')

In [163]:
df_actions_1.head()
df_actions_2.head()
df_actions_3.head()
df_actions_4.head()
df_actions_5.head()

,agg,bil,btcusd,gld,spy,vb,vnq,vo,vwo
date,,,,,,,,,
2023-04-05,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
2023-04-06,0.171251,0.171251,0.171251,0.171251,0.063000,0.063000,0.063000,0.063000,0.063000
2023-04-10,0.171251,0.171251,0.171251,0.171251,0.063000,0.063000,0.063000,0.063000,0.063000
2023-04-11,0.171251,0.171251,0.171251,0.171251,0.063000,0.063000,0.063000,0.063000,0.063000
2023-04-12,0.171251,0.171251,0.171251,0.171251,0.063000,0.063000,0.063000,0.063000,0.063000


In [164]:
df_actions_1.to_csv('df_actions_1.csv')
df_actions_2.to_csv('df_actions_2.csv')
df_actions_3.to_csv('df_actions_3.csv')
df_actions_4.to_csv('df_actions_4.csv')
df_actions_5.to_csv('df_actions_5.csv')

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [165]:
from pyfolio import timeseries

return_dfs = [df_daily_return_1, df_daily_return_2, df_daily_return_3, df_daily_return_4, df_daily_return_5]
agent_names = ['A2C', 'DDPG', 'PPO', 'SAC', 'TD3']

for i, df_return in enumerate(return_dfs):
    strat = convert_daily_return_to_pyfolio_ts(df_return)
    stats = timeseries.perf_stats(strat)
    print(f"\nPerformance Stats for {agent_names[i]}:\n")
    print(stats)



Performance Stats for A2C:

Annual return          0.170588
Cumulative returns     0.371134
Annual volatility      0.137816
Sharpe ratio           1.211785
Calmar ratio           1.247506
Stability              0.918375
Max drawdown          -0.136743
Omega ratio            1.229136
Sortino ratio          1.872602
Skew                   0.489047
Kurtosis               5.805193
Tail ratio             1.222568
Daily value at risk   -0.016700
dtype: float64

Performance Stats for DDPG:

Annual return          0.178419
Cumulative returns     0.389577
Annual volatility      0.128544
Sharpe ratio           1.341581
Calmar ratio           1.585477
Stability              0.927624
Max drawdown          -0.112533
Omega ratio            1.254921
Sortino ratio          2.076112
Skew                   0.387732
Kurtosis               4.785854
Tail ratio             1.164639
Daily value at risk   -0.015511
dtype: float64

Performance Stats for PPO:

Annual return          0.145604
Cumulative returns